In [1]:
# Load extensions
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [2]:
# Importeren van bibliotheken
import math
import numpy as np
import matplotlib.pyplot as plt

from berekeningen_andreas import Brug

# Zoeken van meest ideale parameters voor brug
## Exploratie van formules
### Probleemstelling
Berekeningen gaan uit van volgende situatie:
![Berekeningen](figures/berekeningen_andreas.svg)

In [3]:
brug = Brug(2, 0.2, 0.1, 0.1, 3)
brug.__dict__

{'h1': 2,
 'h2': 0.2,
 'd1': 0.1,
 'd2': 0.1,
 'N': 3,
 'cosa': 0.6,
 'sina': 0.7999999999999999,
 'cosb': 0.9912279006826348,
 'sinb': 0.13216372009101796}

In [4]:
Va, Vb, Fi = brug.calc_reactie_krachten()
Va, Vb, Fi

(200.0, 200.0, [666.6666666666667, 533.3333333333333, 400.0])

In [5]:
FiA, FiB = brug.calc_touw_kracht()
FiA, FiB

((925.9259259259262, 740.7407407407406, 555.5555555555557),
 (-560.4720722378354, -448.3776577902682, -336.28324334270116))

In [6]:
brug.calc_kant()

(-7407.407407407407,
 -1333.3333333333333,
 (555.5555555555557, 2996.1108764676565, 4456.926915584795),
 740.7407407407405,
 1333.3333333333333,
 (-555.5555555555557, -534.1557445131835, -555.5555555555554))

### Wegdek

In [7]:
x = np.arange(0, 2, 0.001)
N, D, M = list(zip(*[brug.calc_intern_wegdek(xi) for xi in x]))

plt.figure()
ax1 = plt.subplot(3,1,1)
ax1.plot(x, N)
ax2 = plt.subplot(3,1,2, sharex=ax1)
ax2.plot(x, D)
ax3 = plt.subplot(3,1,3, sharex=ax1)
ax3.plot(x, M)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Piloon 1

In [8]:
x = np.arange(0, brug.h1, 0.001)
N, D, M = list(zip(*[brug.calc_intern_balk(xi, True) for xi in x]))

plt.figure()
ax1 = plt.subplot(3,1,1)
ax1.plot(x, N)
ax1.set_ylabel("N")
ax2 = plt.subplot(3,1,2)
ax2.plot(x, D)
ax2.set_ylabel("D")
ax3 = plt.subplot(3,1,3)
ax3.plot(x, M)
ax3.set_ylabel("M")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'M')

### Piloon 2

In [9]:
x = np.arange(0, brug.h2, 0.001)
N, D, M = list(zip(*[brug.calc_intern_balk(xi, False) for xi in x]))

plt.figure()
ax1 = plt.subplot(3,1,1)
ax1.plot(x, N)
ax1.set_ylabel("N")
ax2 = plt.subplot(3,1,2)
ax2.plot(x, D)
ax2.set_ylabel("D")
ax3 = plt.subplot(3,1,3)
ax3.plot(x, M)
ax3.set_ylabel("M")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'M')

## Optimalisatie

In [10]:
# Maxwaarden
h1_max = 2 # 2m
h2_max = 2 # 2m
h1_min = 0.06 # 6cm
h2_min = 0.06 # 6cm
d1_max = 2 # 2m
d2_max = 2 # 2m
d1_min = 0.1 # 10cm
d2_min = 0.1 # 10cm
N_max = 10
N_min = 1

In [11]:
def clamp(x, mini, maxi):
    return min(maxi, max(mini, x))

def genRandomBrug():
    from random import random, randint
    return Brug(
        h1_min + random()*(h1_max - h1_min),
        h2_min + random()*(h2_max - h2_min),
        d1_min + random()*(d1_max - d1_min),
        d2_min + random()*(d2_max - d2_min),
        randint(N_min, N_max),
    )

def genFromParents(p1, p2):
    import random
    h1 = (p1.h1 if random.choice([True, False]) else p2.h1) + random.gauss(0,1)
    h2 = (p1.h2 if random.choice([True, False]) else p2.h2) + random.gauss(0,1)
    d1 = (p1.d1 if random.choice([True, False]) else p2.d1) + random.gauss(0,1)
    d2 = (p1.d2 if random.choice([True, False]) else p2.d2) + random.gauss(0,1)
    N = (p1.N if random.choice([True, False]) else p2.N) + int(random.gauss(0,N_max/2))
    
    return Brug(
        clamp(h1, h1_min, h1_max),
        clamp(h2, h2_min, h2_max),
        clamp(d1, d1_min, d1_max),
        clamp(d2, d2_min, d2_max),
        clamp(N, N_min, N_max),
    )

In [12]:
def kill_population(population):
    scores = [[brug.calc_fitness(), brug] for brug in population]
    scores = sorted(scores, key=lambda score: score[0])
    # delete half the population #Thanos-feeling
    scores = scores[:len(scores)//2]
    return list(zip(*scores))[1]

def regen_population(population):
    new_p = [genFromParents(population[i], population[i+1]) for i in range(0, len(population), 2)]
    return [*population,
            *new_p,
            *[genRandomBrug() for _ in range(len(population)//2)]]

In [13]:
population = [genRandomBrug() for i in range(100)] # Populatie van 100

In [14]:
for i in range(100): # 1000 generations
    print('.', end="")
    population = kill_population(population)
    population = regen_population(population)

....................................................................................................

In [15]:
def find_best(population):
    scores = [[brug.calc_fitness(), brug] for brug in population]
    scores = sorted(scores, key=lambda score: score[0])
    return scores[0]

def print_brug(brug):
    x = np.arange(0, max([brug.h1, brug.h2, 2]), 0.001)
    N1, _, _ = list(zip(*[brug.calc_intern_balk(xi, True) for xi in x]))
    N2, _, _ = list(zip(*[brug.calc_intern_balk(xi, False) for xi in x]))
    _, D, _ = list(zip(*[brug.calc_intern_wegdek(xi) for xi in x]))
    
    fig = plt.figure()
    ax1 = plt.subplot(3, 1, 1)
    ax1.plot(x, N1)
    ax1.set_ylabel("N1")
    ax2 = plt.subplot(3, 1, 2)
    ax2.plot(x, N2)
    ax2.set_ylabel("N2")
    ax3 = plt.subplot(3, 1, 3)
    ax3.plot(x, D)
    ax3.set_ylabel("D")
    
    print("======= Info Brug ========")
    print(f"h1:\t{brug.h1}")
    print(f"h2:\t{brug.h2}")
    print(f"d1:\t{brug.d1}")
    print(f"d2:\t{brug.d2}")
    print(f"N:\t{brug.N}")
    
    print("Krachten in touwen A")
    for F in brug.FiA:
        print(f"\t{F}")
    print("Krachten in touwen B")
    for F in brug.FiB:
        print(f"\t{F}")
    print("Krachten in touwen C")
    for F in brug.FiC:
        print(f"\t{F}")
    print("Krachten in touwen D")
    for F in brug.FiD:
        print(f"\t{F}")
    print("Maximale kracht trek/druk:")
    print(f"\tN1: {np.max(np.abs(N1))}")
    print(f"\tN2: {np.max(np.abs(N2))}")
    print(f"\tD: {np.max(np.abs(D))}")
    
    return fig

In [16]:
best = find_best(population)

In [17]:
print_brug(best[1])
print(f"Score brug: {best[0]}")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

======= Info Brug ========
h1:	2
h2:	0.06
d1:	2
d2:	0.1
N:	2
Krachten in touwen A
	660.8112303255878
	660.8112303255878
Krachten in touwen B
	-366.92306545345394
	-366.92306545345394
Krachten in touwen C
	366.55211912943855
	409.8177278350128
Krachten in touwen D
	-366.5521191294386
	-382.69164752020333
Maximale kracht trek/druk:
	N1: 1549.5990836197016
	N2: 266.6666666666667
	D: 666.4000000000001
Score brug: 4019.610538373424


### Enkele berekende bruggen
```
======= Info Brug ========
h1:	0.06
h2:	1.0652839017460745
d1:	0.1
d2:	2
N:	2
Krachten in touwen A
	-708.0892728345934
	-708.0892728345934
Krachten in touwen B
	905.4218833682901
	905.4218833682901
Krachten in touwen C
	-707.3734194648738
	-738.5195315469434
Krachten in touwen D
	707.3734194648738
	732.0296004492964
Maximale kracht trek/druk:
	N1: 266.6666666666666
	N2: 1585.3853201549225
	D: 666.4000000000001
Score brug: 3856.0163142124156
```
```
======= Info Brug ========
h1:	1.126803803965232
h2:	0.06
d1:	2
d2:	0.1
N:	2
Krachten in touwen A
	872.737237017732
	872.737237017732
Krachten in touwen B
	-667.255538772807
	-667.255538772807
Krachten in touwen C
	666.5809668731615
	692.5245096390946
Krachten in touwen D
	-666.5809668731615
	-695.9309607161756
Score brug: 3875.5966914564374
```
```
======= Info Brug ========
h1:	0.06
h2:	1.2516051534550352
d1:	0.1
d2:	2
N:	2
Krachten in touwen A
	-597.3713229720112
	-597.3713229720112
Krachten in touwen B
	818.4997111963966
	818.4997111963966
Krachten in touwen C
	-596.7674015585268
	-623.0434586797123
Krachten in touwen D
	596.7674015585268
	625.2992554442243
Maximale kracht trek/druk:
	N1: 266.6666666666666
	N2: 1573.7716882747561
	D: 666.4000000000001
Score brug: 3909.1072028074623
```
```
======= Info Brug ========
h1:	1.572067987257989
h2:	0.06
d1:	2
d2:	0.1
N:	2
Krachten in touwen A
	727.0758927725531
	727.0758927725531
Krachten in touwen B
	-470.76636300630054
	-470.76636300630054
Krachten in touwen C
	470.2904347579321
	505.30783775277274
Krachten in touwen D
	-470.290434757932
	-490.997628708161
Score brug: 3969.8161468717476
```
```
======= Info Brug ========
h1:	0.8867440380990106
h2:	0.06
d1:	2
d2:	0.1
N:	2
Krachten in touwen A
	1032.9862077900755
	1032.9862077900755
Krachten in touwen B
	-861.0049957136215
	-861.0049957136215
Krachten in touwen C
	860.1345499221471
	881.0165881426548
Krachten in touwen D
	-860.1345499221471
	-898.0068340091038
Maximale kracht trek/druk:
	N1: 1601.4252388529362
	N2: 266.6666666666667
	D: 666.4000000000001
Score brug: 3980.4220933118136
```
```
======= Info Brug ========
h1:	2
h2:	0.06
d1:	2
d2:	0.1
N:	2
Krachten in touwen A
	660.8112303255878
	660.8112303255878
Krachten in touwen B
	-366.92306545345394
	-366.92306545345394
Krachten in touwen C
	366.55211912943855
	409.8177278350128
Krachten in touwen D
	-366.5521191294386
	-382.69164752020333
Maximale kracht trek/druk:
	N1: 1549.5990836197016
	N2: 266.6666666666667
	D: 666.4000000000001
Score brug: 4019.610538373424
```